In [1]:
import numpy as np
import pandas as pd
import sklearn
import scipy as s
from IPython.display import display
import matplotlib.pyplot as plt

# Table of Contents

# Introduction

Ranking systems have many different applications, however most common ones are poor as they rely on arbitrary conventions which lead to poor proformance. In general, they try to answer they question **What is the probability that player 1 defeats player 2?**. In order to determine this probabilistically, there are a number of things to consider:

- Considers who you played against.
- Must be robust against players who have not played against eachother.
- Give a good estimate at any point in the season.
- Take into account performance inconsistancy.

Therefore, what we want to infer is the player's **skill**, $w$. These skills must be comparable (i.e. a player with a higher skill must be more likely to win), and as such we want to do a probabilstic inference of a player's skill and be able to compute the probability of a game's outcome.

### A generative model for skill

A summary of a generative model for game outcomes can be defined as:

1. **Skills** Take two players with known skills, $$w_i \in \mathbb{R}$$
2. **Skill Difference**: $$s = w_1 - w_2$$
3. **Performance Difference**: Add noise ($n \sim \mathcal{N}(0, 1))$ to account for performance inconsistance: $$t = s + n$$
4. **Game outcome** is given by $y=sign(t)$:
    - $y = +1$ means player 1 wins
    - $y = -1$ means player 2 wins

![FactorGraph](Figures/FactorGraph.jpg)

### The probability of a player winning given their skills
    
Therefore we can work out the probability that player 1 wins given the players skills':

$$p(y| w_1, w_2) = \int \int p(y|t) p(t|s) p(s | w_1, w_2) dsdt$$
$$ = \int p(y|t) p(t|w_1, w_2) dt$$
$$ = \int^{+\infty}_{-\infty} \delta(y - sign(t)) \mathcal{N}(t; w_1 - w_2, 1) dt$$
$$ = \int^{+\infty}_{-\infty} \delta(1 - sign(yt)) \mathcal{N}(yt; y(w_1 - w_2), 1) dt $$
set z = yt and note the change in limits and dt:
$$ = \int^{+y\infty}_{-y\infty} \delta(1 - sign(z)) \mathcal{N}(z; y(w_1 - w_2), 1) y dz $$
$$ = \int^{+\infty}_{-\infty} \delta(1 -sign(z)) \mathcal{N}(z; y(w_1 - w_2), 1) dz $$
And rearrange the limits:
$$ = \int^{+\infty}_{0} \mathcal{N}(z; y(w_1 - w_2), 1) dz $$
using $x = y(w_1-w_2) - z$
$$ = \int^{y(w_1 - w_2)}_{-\infty} \mathcal{N}(x; 0, 1) dx $$
$$ = \Phi(y(w_1 - w_2))$$

where $\Phi(a)$ is the gaussian c.d.f, or the *probit* function.

For the probability of player 1 winning, we simply use $p(y=1| w_1, w_2) = p(t>0| w_1, w_2) = \Phi(w_1 - w_2)$

![PerformanceDifference](Figures/PerformanceDifference.jpg)

### Computing the skills (the posterior)

$$p(w_1, w_2 | y) = \frac{Priors \times Likelihood}{Evidence}$$
<br>
$$ = \frac{p(w_1)p(w_2) \times p(y|w_1, w_2)}{\int \int p(w_1)p(w_2) \times p(y|w_1, w_2) dw_1 dw_2}$$
<br>
$$ = \frac{\mathcal{N}(w_1; \mu_1, \sigma_1^2) \mathcal{N}(w_2; \mu_2, \sigma_2^2) \times \Phi(y(w_1 - w_2))}{\int \int \mathcal{N}(w_1; \mu_1, \sigma_1^2) \mathcal{N}(w_2; \mu_2, \sigma_2^2) \times \Phi(y(w_1 - w_2)) dw_1 dw_2}$$

The joint posterior over skills does not have a closed form as the probit function is not closed. Additionally, $w_1$ and $w_2$ have become correlated due to the priors and therefore does not factorise, nor is it a gaussian density function.

Fortunately, the evidence does have a closed form:

$$p(y) = \int \int \mathcal{N}(w_1; \mu_1, \sigma_1^2) \mathcal{N}(w_2; \mu_2, \sigma_2^2) \times \Phi(y(w_1 - w_2)) dw_1 dw_2 = \Phi \bigg(\frac{y(\mu_1 - \mu_2)}{\sqrt{1 + \sigma_1^2 + \sigma_2^2}} \bigg)$$

This is effectively a smoother version of the likelihood as we are using mean skills of each player, and normalising over their variances.

### Computing the Conditionals

For notation we have have $g = \{1, ..., G\}$ games where $I_g$ is the id of player 1, and $J_g$ is the id of player 1. The outcome of game g is:

$$y_g = \bigg{\{} \begin{array}{c} +1 \; if \; I_g \; wins \\ -1 \; if \; J_g \; wins \end{array}$$

Note also that the vector of skills is $\boldsymbol{w} = [w_1, ..., w_d]^T$, (d=107 players) and the vector of performance differences is $\boldsymbol{t} = [t_1, ..., t_G]^T$.

In **True Skill** the joint distribution $p(w_{1:d}, t_{1:G})$ is intractable. However, the two conditionals are tractable and easily derived to be:

- **Performance given skill difference**: $p(t_g | w_{I_g}, w_{J_g}, y_g) \propto \delta(y_g - sign(t_g)) \mathcal{N}(t_g;  w_{I_g} - w_{J_g}, 1)$
- **Skills given performances**: $p( \boldsymbol{w} | \boldsymbol{t}, \boldsymbol{y}) = p( \boldsymbol{w} | \boldsymbol{t}) \propto p(\boldsymbol {w}; \boldsymbol{\mu}_0, \Sigma_0) \prod_{g=1}^G \mathcal{N}(\boldsymbol{w}; \boldsymbol{\mu}_g, \Sigma_g)$

In order to calculate these, we can take the performance difference directly from the derivation of the evidence:

$$p(t | w_1, w_2, y) = p(y|t) p(t|w_1, w_2) = \delta(y - sign(t)) \mathcal{N}(t; w_1 - w_2, 1) dt$$

and similarly, we note that the priors for skill are $p(w_i) = \mathcal{N}(w_i | u_i, \sigma_i^2) \implies p(\boldsymbol{w}) = \prod_{i=1}^d p(w_i) = \mathcal{N}(\boldsymbol{w} ; \boldsymbol{\mu}_0, \Sigma_0)$. Thus,

$$p( \boldsymbol{w} | \boldsymbol{t}) = \frac{p(\boldsymbol{w})p(\boldsymbol{t}| \boldsymbol{w})}{p(\boldsymbol{t})} \propto p(\boldsymbol{w}) \prod_{g=1}^G p(t_g | w_{I_g}, w_{J_g}) = \mathcal{N}(\boldsymbol{w} ; \boldsymbol{\mu}_0, \Sigma_0) \prod_{g=1}^G \mathcal{N}(\boldsymbol{w}; \boldsymbol{\mu}_g, \Sigma_g)
$$

# Load the Data

In [2]:
import scipy.io
mat = scipy.io.loadmat('tennis_data.mat')

In [3]:
PLAYERS = pd.DataFrame(mat['W'], columns=['name']).applymap(lambda x: x[0])  # remove the list around each players name
GAMES = pd.DataFrame(mat['G'], columns=['winner', 'loser'])
display(PLAYERS.head(), GAMES.head())
display(len(PLAYERS), len(GAMES))

,name
0,Rafael-Nadal
1,Juan-Monaco
2,Juan-Martin-Del-Potro
3,Mardy-Fish
4,Roger-Federer


,winner,loser
0,1,2
1,1,3
2,1,3
3,1,3
4,1,3


107

1801

# Gibbs Rank

### Integration of an Intractable Posterior

The basis of Monte-Carlo approximation is:
$$\mathbb{E}_{p(x)} \big[ \Phi(x) \big] \approx \hat{\Phi} = \frac{1}{T} \sum^t_{\tau = 1} \Phi(x^{(\tau)}), \text{  where } x^{(\tau)} \sim p(x)$$

Note that $x^{(\tau)}$ is the $\tau$th d-dimensional sample from the distribution p(x), which is analytically intractable (and typically d>>1).

This is infact an unbiased estimate, with $Var[\hat{\Phi}] = \frac{Var[\Phi]}{T}$. Note $Var[\Phi] = \int (\Phi(x) - \mathbb{E}[\Phi])^2 p(x) dx$. Note that this is independent of dimension d, of x.


How do we generate samples from p(x)? If the distribution has a standard form then we could generate independent samples, however, it is often difficult to sample from this joint distribution if it is within a high dimensional space (the curse of dimensionality). In order to get around this we can use **Gibbs Sampling**, which uses a Markov Chain to generate dependent samples from the desired distribution:

$$x_i' \sim  p(x_i | x_{1, 2, ..., i-1, i+1, ..., d})$$

### Gibbs Sampling

Gibbs sampling assumes that we know the forms of the joint and conditional distributions. Given that $p(x, y) \propto p(x|y)$, we can thus sample from the conditional distributions. E.g. A number of experiments, k, are performed where we flip a coin n times (unknown). The coin is biased such that $p(heads) = \theta$.

- $n \sim U(5, 8)$
- $\theta \sim U(0, 1) = beta(1, 1)$

This gives us a sequence with the number of heads that were thrown: $X = (x_1, x_2, .., x_k)$. We are aiming to find $p(n, \theta | X) \propto p(X | n, \theta)p(n, \theta) = Zp(X | n, \theta)$. 

$$p(n, \theta | X) \propto \prod^k_{i=1} C^n_{x_i} \theta^{x_i} (1-\theta)^{n-x_i}$$
$$= \theta^{\sum+_{i=1}^k x_i}(1-\theta)^{\sum_{i=1}^k n-x_i} \prod_{i=1}^k  C^n_{x_i}$$
$$\theta^k\hat{x}(1-\theta)^{k(n-\hat{x})} \prod_{i=1}^k  C^n_{x_i}$$

Above is the joint probability upto a constant of proportionality. Next we need to find the conditionals, which we can do by simply splitting the above into $f_1(n)f_2(\theta)$.

$$p(\theta |n, X) \propto \theta^k\hat{x}(1-\theta)^{k(n-\hat{x})} = beta(k\hat{x}+1, k(n-\hat{x})+1)$$
$$p(n |\theta, X) \propto (1-\theta)^{kn}\prod_{i=1}^k  C^n_{x_i}$$

Note that there is no "nice" form for $p(n | \theta, X)$ here. But what we can do, is we know $n \in \{5, 6, 7, 8\}$, so we can use these values and sum them up to get a categorical distribution by normalising with them.

Gibbs sampling here then works simply by choosing some random starting point and then iteratively sampling from each of these two conditionals.

For **our problem**, we have 107 different players, and hence are sampling from 107 different conditional distributions for thier skills. Consider the distribution $p(x) = p(x_{1:d})$. The gibbs sampling algorithm is as follows:


> 1. Initialise $\{x_i : i = 1, .., d\}$
> 2. For $\tau = 1, ..., T$: <br>
    > &nbsp;&nbsp;&nbsp;&nbsp; For $i = 1, ..., d$: <br>
        > &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Sample $x_{i}^{\tau+1} \sim p(x_i|x_{\backslash i}^{(\tau)})$
        
**Try over-relaxation?**

![GibbsSampling](Figures/GibbsSampling.jpg)

Above is an illustration of Gibbs sampmling by alternate updates of two variables, whose distribution is a correlated GAussian. The step size is govened by the standard deviation of the  <span style="color:green">condiditional distribution</span>, and is O(l), leading to slow progression in the direction of elongation of the <span style="color:red">joint distribution</span>. The number of steps needed to obtain an independent sample from the distribution is $O((\frac{L}{l})^2)$. I.e. **Strong correlations slow down gibbs sampling**.

In **True Skill** the joint distribution $p(w_{1:d}, t_{1:G})$ is intractable. However, the two conditionals are tractable and have already been derived to be:

- **Performance given skill difference**: $p(t_g | w_{I_g}, w_{J_g}, y_g) = \frac{p(y|t) p(t|w_1, w_2)}{p(y)} \propto \delta(y_g - sign(t_g)) \mathcal{N}(t_g;  w_{I_g} - w_{J_g}, 1)$
- **Skills given performances**: $p( \boldsymbol{w} | \boldsymbol{t}, \boldsymbol{y}) = p( \boldsymbol{w} | \boldsymbol{t}) \propto p(\boldsymbol{w}) \prod_{g=1}^G p(t_g | w_{I_g}, w_{J_g})  =  p(\boldsymbol {w}; \boldsymbol{\mu}_0, \Sigma_0) \prod_{g=1}^G \mathcal{N}(\boldsymbol{w}; \boldsymbol{\mu}_g, \Sigma_g)$

We can describe the two above distributions in more detail:

#### For the conditional posterior of skills (skills given performances) we get:

$$p(t_g|w_{I_g}, w_{J_g}) = \mathcal{N}(t_g; (w_{I_g} - w_{J_g}), 1) \propto exp(-\frac{1}{2}(t_g-(w_{I_g} - w_{J_g}))^2 )$$

renaming $t_g = \mu_1 - \mu_2$

$$\propto exp(-\frac{1}{2}((w_{I_g} - \mu_1) - ( w_{J_g} - \mu_2) )^2 )$$

$$ \propto exp \bigg( -\frac{1}{2} 
\left[\begin{array}{c} w_{I_g} - \mu_1 \\ w_{J_g} - \mu_2 \end{array}\right]
\left[\begin{array}{cc} 1 & -1 \\ -1 & 1 \end{array}\right]
\left[\begin{array}{c} w_{I_g} - \mu_1 \\ w_{J_g} - \mu_2 \end{array}\right] \bigg)
$$

Recall that when multiplying gaussians, the precisions add up and the means weighted by the precisions also add up:

- $\Sigma_c^{-1} = \Sigma_a^{-1} + \Sigma_b^{-1}$
- $\mu_c = \Sigma_c(\Sigma_a^{-1}\mu_a +  \Sigma_b^{-1} \mu_b)$

Thus:

$$p(\boldsymbol {w}; \boldsymbol{\mu}_0, \Sigma_0) \prod_{g=1}^G \mathcal{N}(\boldsymbol{w}; \boldsymbol{\mu}_g, \Sigma_g) = \mathcal{N}(\boldsymbol{w}; \mu, \Sigma)$$

Where each game precision, $\Sigma_g^{-1}$ contains only 4 non-zero entries, the full precisions and means are:

- $\Sigma^{-1} = \Sigma_0^{-1} + \sum_{g=1}^G \Sigma_g^{-1}  = \Sigma_0^{-1} + \tilde{\Sigma}^{-1}$
- $\mu = \Sigma(\Sigma_0^{-1}\mu_0 + \sum_{g=1}^G \Sigma_g^{-1}\mu_g) = \Sigma(\Sigma_0^{-1}\mu_0 + \tilde{\mu})$

defining these further:

$$\tilde{\mu}_i = \sum_{g=1}^G t_g(\delta(i-I_g) - \delta(i-J_g))$$


$$ [ \tilde{\Sigma}^{-1} ]_{ij}  = \bigg\{  \begin{array}{c} \sum_{g=1}^G \delta(i-I_g) + \delta(i-J_g) \; \; \; for \; i = j \\ \sum_{g=1}^G \delta(i-I_g)\delta(j-J_g) + \delta(i-J_g)\delta(j-I_g) \; \; \; for \; i \neq j  \end{array} $$

The conditional skills can be jointly sampled directly using the cholesky factorization.

#### For the conditional posterior of the performances (performance given skill difference), 

we get a truncated gaussian that needs no further analysis, however the question becomes how do we sample from it? The inverse transformation method, or rejection sampling are both possibilities.

todo. 
1. Read the gibbs section of Pattern Recog and add anything from there into here
1. Reread the notes and add the specific maths
1. implement the gibbs sampling algorithm

In [5]:
def gaussian_distribution(x, mu, s):
    exponent = -0.5*((x-mu)/s)**2
    Z = (s * np.sqrt(2*np.pi))
    return np.exp(exponent) / Z

In [6]:
def gibbs_sampler(total_iters, players=PLAYERS, games=GAMES):
     skills_container = np.zeros(len(), total_iters)